Dask + Multi processing

In [1]:
from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da
import dask_ml
import joblib
import numpy as np
import pandas as pd 
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression
from sklearn import tree
import time
import warnings
warnings.filterwarnings('ignore')
import random
from xgboost import XGBClassifier
# import shutil
# import os
from our_evaluate_sepsis_score import evaluate_performance_dask
import our_functions_library as flib

In [2]:
# only run this once
client = Client()

In [3]:
client

Client Scheduler: tcp://127.0.0.1:58384 Dashboard: http://127.0.0.1:58383/status,Cluster Workers: 4 Cores: 16 Memory: 16.95 GB


In [4]:
# client.shutdown()
#client.close()

 # Yeah, you may start coding from here

In [93]:
filename = '../../raw_data/raw_data_5000.csv' # use raw dataset
originalData = dd.read_csv(filename,dtype={'Gender':bool,'SepsisLabel':bool}) # read csv data into Dask DataFrame
Uniq_ID= np.unique(originalData['Patient_id'])
print("Data size:",originalData.shape)
print('Patient id size:',len(Uniq_ID))
print(Uniq_ID)

X_full_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_full_columns = ['Patient_id', 'SepsisLabel']

# Initialize the empty array X_train, X_test, y_train, y_test
# X_train = dd.from_pandas(pd.DataFrame(columns = X_columns),npartitions=2)
# X_test = dd.from_pandas(pd.DataFrame(columns = X_columns),npartitions=2)
# y_train = dd.from_pandas(pd.DataFrame(columns = y_columns),npartitions=2)
# y_test = dd.from_pandas(pd.DataFrame(columns = y_columns),npartitions=2)

# Below are the lists for KNN results of different K value
KNN_UtilityScore_mean = []
KNN_UtilityScore_std = []
KNN_F1Score_mean = []
KNN_F1Score_std = []
KNN_auroc_mean = []
KNN_auprc_mean = []
KNN_accuracy_mean = []
KNN_accuracy_std = []
KNN_positiveprediction_mean = []
KNN_baseline_mean = [ ]
KNN_total_Time= [ ]
fillmethod =""  

Data size: (Delayed('int-81fb71b3-5549-4ff4-88fc-fb2752b054ee'), 43)
Patient id size: 5000
[   1    2    3 ... 4998 4999 5000]


In [94]:
print("Original Data set size:",originalData.compute().shape)
print(originalData.memory_usage(deep=True).sum().compute(),"bytes")

Original Data set size: (192206, 43)
5064968 bytes


In [95]:
missing = originalData.isna().sum()/len(originalData)*100
print(missing.compute().sort_values(ascending=False))

Bilirubin_direct    99.791890
Fibrinogen          99.300230
TroponinI           99.004193
Bilirubin_total     98.492763
Alkalinephos        98.363214
AST                 98.349687
Lactate             97.238900
PTT                 96.973039
SaO2                96.389811
EtCO2               96.218640
Phosphate           95.987118
HCO3                95.779528
Chloride            95.436667
BaseExcess          94.290501
PaCO2               94.252000
Calcium             94.161993
Platelets           94.042850
Creatinine          93.902896
Magnesium           93.632873
WBC                 93.559514
BUN                 93.078260
pH                  92.786906
Hgb                 92.523647
FiO2                91.691727
Hct                 91.005484
Potassium           90.563250
Glucose             82.698771
Temp                66.309064
Unit1               38.018064
Unit2               38.018064
DBP                 31.303393
Resp                15.868911
SBP                 14.752921
O2Sat     

In [96]:
leftAllColumns = list(originalData.columns).copy()
almostMissingColumns = list(missing[missing>91].index)
# print(almostMissingColumns)

In [97]:
#!!!Only run this cell if you don't want those missing columns in your whole training
# leftAllColumns =  [i for i in leftAllColumns if i not in almostMissingColumns]

In [98]:
# print(leftAllColumns)
trainColumns = leftAllColumns.copy()
# trainColumns.remove('time')
trainColumns.remove('SepsisLabel')
# print(missing.compute())

In [99]:
print(len(leftAllColumns),"Columns left after deleting 90:   ",leftAllColumns,"\n")
print(len(trainColumns),"Columns for training algorithm:   ",trainColumns)

43 Columns left after deleting 90:    ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2', 'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct', 'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC', 'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS', 'SepsisLabel', 'Patient_id', 'time'] 

42 Columns for training algorithm:    ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2', 'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct', 'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC', 'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS', 'Patient_id', 'time']


In [100]:
# originalData = originalData[leftAllColumns]
print(originalData.head())

      HR  O2Sat  Temp    SBP    MAP  DBP  Resp  EtCO2  BaseExcess  HCO3  ...  \
0    NaN    NaN   NaN    NaN    NaN  NaN   NaN    NaN         NaN   NaN  ...   
1   97.0   95.0   NaN   98.0  75.33  NaN  19.0    NaN         NaN   NaN  ...   
2   89.0   99.0   NaN  122.0  86.00  NaN  22.0    NaN         NaN   NaN  ...   
3   90.0   95.0   NaN    NaN    NaN  NaN  30.0    NaN        24.0   NaN  ...   
4  103.0   88.5   NaN  122.0  91.33  NaN  24.5    NaN         NaN   NaN  ...   

   Platelets    Age  Gender  Unit1  Unit2  HospAdmTime  ICULOS  SepsisLabel  \
0        NaN  83.14   False    NaN    NaN        -0.03       1        False   
1        NaN  83.14   False    NaN    NaN        -0.03       2        False   
2        NaN  83.14   False    NaN    NaN        -0.03       3        False   
3        NaN  83.14   False    NaN    NaN        -0.03       4        False   
4        NaN  83.14   False    NaN    NaN        -0.03       5        False   

   Patient_id  time  
0           1     0  


In [101]:
originalData

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,Patient_id,time
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,int64,bool,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [102]:
originalData = flib.linearFillingAllSeparate(originalData.compute(),Uniq_ID)
#data, fillmethod = flib.linearFillingAll(originalData)#
#data = flib.linearFillingAll(originalData,1)#fill other missing data with 0
#data = flib.linearFillingAll(originalData,2,)#fill other missing data with overall average value
# data = flib.linearFillingAll(originalData,0,True)#forwardfilling
#data = flib.linearFillingAll(originalData,1, True)#forwardfilling + fill other missing data with 0
# originalData,fillingmethod = flib.linearFillingAll(originalData.compute(),2,True)#forwardfilling + fill other missing data with overall average value
#oridata after linearly imputation is filled, so oridata needs to be generated again
#print(originalData)

Filling using Linear interpolation
Fill with missing column with healty/sepsis mean
HR                  0.0
O2Sat               0.0
Temp                0.0
SBP                 0.0
MAP                 0.0
DBP                 0.0
Resp                0.0
EtCO2               0.0
BaseExcess          0.0
HCO3                0.0
FiO2                0.0
pH                  0.0
PaCO2               0.0
SaO2                0.0
AST                 0.0
BUN                 0.0
Alkalinephos        0.0
Calcium             0.0
Chloride            0.0
Creatinine          0.0
Bilirubin_direct    0.0
Glucose             0.0
Lactate             0.0
Magnesium           0.0
Phosphate           0.0
Potassium           0.0
Bilirubin_total     0.0
TroponinI           0.0
Hct                 0.0
Hgb                 0.0
PTT                 0.0
WBC                 0.0
Fibrinogen          0.0
Platelets           0.0
Age                 0.0
Gender              0.0
Unit1               0.0
Unit2               0.0
Hosp

In [103]:
if (originalData.isnull().values.any()):
    print('Data isn´t all filled before K-Fold Func, there are still missing values...')
originalData = dd.from_pandas(originalData,npartitions = 10)

In [104]:
originalData.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,Patient_id,time
0,97.0,95.0,36.11,98.0,75.330,63.97,19.0,33.41,24.000000,45.0,...,317.0,83.14,False,0.49,0.51,-0.03,1,False,1,0
1,97.0,95.0,36.11,98.0,75.330,63.97,19.0,33.41,24.000000,45.0,...,317.0,83.14,False,0.49,0.51,-0.03,2,False,1,1
2,89.0,99.0,36.11,122.0,86.000,63.97,22.0,33.41,24.000000,45.0,...,317.0,83.14,False,0.49,0.51,-0.03,3,False,1,2
3,90.0,95.0,36.11,122.0,88.665,63.97,30.0,33.41,24.000000,45.0,...,317.0,83.14,False,0.49,0.51,-0.03,4,False,1,3
4,103.0,88.5,36.11,122.0,91.330,63.97,24.5,33.41,23.857143,45.0,...,317.0,83.14,False,0.49,0.51,-0.03,5,False,1,4


In [105]:
#Shuffle the id set 
#This block will shuffle the id sets of patients in a fix manner, so for every time you run the code, the dataset and trainset are always the same.
seed = 2
random.seed(seed)
if(np.array_equal(range(1,21),Uniq_ID[:20]) ):
    print("The original uniq id set is:\n",Uniq_ID[0:20])
    print("The first 20 patient ids are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.")   
    random.shuffle(Uniq_ID)# randomly sorted the patient IDs 
#     Uniq_ID = da.from_array(Uniq_ID)
else:
    print("They are not the same. Uniq_id has already been shuffled.")    
print("The fixed shuffelld id set, it should be 211 275 153...\n",Uniq_ID[:20])

The original uniq id set is:
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
The first 20 patient ids are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.
The fixed shuffelld id set, it should be 211 275 153...
 [1424  383 2649 1711 1337 1952  932 4825 1044 1220 2847 1507  214 4122
 1005 3831  620  541 4779 2414]


In [106]:
def KNN_reset():
    KNN_UtilityScore_mean.clear()
    KNN_UtilityScore_std.clear()
    KNN_F1Score_mean.clear()
    KNN_F1Score_std.clear()
    KNN_auroc_mean.clear()
    KNN_auprc_mean.clear()
    KNN_accuracy_mean.clear()
    KNN_accuracy_std.clear()
    KNN_positiveprediction_mean.clear()
    KNN_baseline_mean.clear()
    KNN_total_Time.clear()
    
def Print_DATA():
    print("Numbers of K:",len(KNN_accuracy_mean))
    print("KNN_UtilityScore_mean", KNN_UtilityScore_mean)
    print("KNN_UtilityScore_std", KNN_UtilityScore_std)
    print("KNN_F1Score_mean", KNN_F1Score_mean)
    print("KNN_F1Score_std", KNN_F1Score_std)
    print("KNN_auroc_mean", KNN_auroc_mean)
    print("KNN_auprc_mean", KNN_auprc_mean)
    print("KNN_accuracy_mean", KNN_accuracy_mean)
    print("KNN_accuracy_std", KNN_accuracy_std)
    print("KNN_positiveprediction_mean", KNN_positiveprediction_mean)
    print("KNN_baseline_mean", KNN_baseline_mean)
    print("KNN_total_time", KNN_total_Time)   
    
def generateTrainDataSet(originalData,test_patienIds, X_columns = X_full_columns, y_columns = y_full_columns):
    train_data= originalData[~(originalData.Patient_id.isin(test_patienIds))]
    X_train = train_data[X_columns]
    y_train = train_data[y_columns] 
    return X_train, y_train


#Function to generate the test dataset
def generateTestDataSet(originalData,patienIds,X_columns = X_full_columns, y_columns = y_full_columns):
    print("test_patienIds: \n", patienIds)
    test_data = originalData[originalData['Patient_id'].isin(patienIds)]
    X_test = test_data[X_columns]
    y_test = test_data[y_columns]
    return X_test, y_test


In [107]:
#Function for 1 fold in Kfold
def OneFold_patient_MP(i,model,KforKNN,idSets,fillmethod = ""): 
    start1 = time.time()
    print("for the",i+1,"th iteration: ")
    X_test,y_test = generateTestDataSet(originalData,idSets[i],X_columns = trainColumns)
    print("Time for splitting id to test dataset:", round(time.time()- start1,3))
    t1 = time.time()
    X_train, y_train = generateTrainDataSet(originalData,idSets[i],X_columns = trainColumns)
    print("Time for splitting id to train dataset:", round(time.time()- t1,3))

    #Now the train and test dataset is generated 
    patientID_ytest = y_test['Patient_id']
    y_test = y_test.drop('Patient_id', 1) 
    y_train = y_train.drop('Patient_id', 1)
    X_train=X_train.astype('float32')
    X_test=X_test.astype('float32') 
    y_train=y_train.astype('bool')
    y_test=y_test.astype('bool')
    print(X_train)
    
    #fill the missing data
    if X_train.isnull().values.any() or X_test.isnull().values.any() :
        t1= time.time()
        print("X_train or X_test contains NaN values, KNN is performed.")
#         X_train, X_test,fillmethod = flib.MeanFilling(X_train,X_test, fillmethod)  
        X_train, X_test, fillmethod = flib.KNNfilling(X_train, X_test, KforKNN)
        print("The type of X_train after KNN filling is",type(X_train))
        print("Time for NaN values filling: ", round(time.time()- t1,2) )
        X_train = dd.from_array(X_train)
        X_test = dd.from_array(X_test)
        if  X_train.isnull().values.any() or X_test.isnull().values.any() :
            print("X_train or X_test still contains NaN values after filling")
    else:
        print("Data set is already full, no more filling is needs")
    print("The type of X_train after KNN filling is",type(X_train))
    
    
    if ( type(model).__name__ == 'LogisticRegression'):  #Scale the data for logistic regression
        t1 = time.time()
        scaler = dask_ml.preprocessing.StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        print("Standard Scaling time: ", round(time.time()- t1,2) )

               
    #fit the model and predict
    with joblib.parallel_backend("dask",scatter=[X_train, y_train]):
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_test).astype(bool)   
        y_predicted_probobility = model.predict_proba(X_test).astype('float32') 
    y_labels = dd.from_pandas(y_test['SepsisLabel'].compute().reset_index(drop=True),npartitions = 2)
    
#     if(i == 1):
#             # create excel writer object
#         writer = pd.ExcelWriter('X_train_90dropped_linearf.xlsx')
#         # write dataframe to excel
#         X_train.compute().to_excel(writer)
#         # save the excel
#         writer.save()
        
    del X_train, X_test, y_train

    print("columns of y_predicted_probobility is: ",len(y_predicted_probobility[0]))
    if(len(y_predicted_probobility[0]) ==2 ):
        y_predicted_probobility =  y_predicted_probobility[:,1].round(4)
    elif(len(y_predicted_probobility[0]) == 1):     
        y_predicted_probobility =  y_predicted_probobility.round(4)
    
    auroc, auprc, physio_accuracy, f_measure, utility_score = evaluate_performance_dask(y_labels.compute(), y_predicted, y_predicted_probobility,patientID_ytest.compute(),idSets[i]) 
    del y_labels, y_predicted_probobility    
    result = { "auroc": round(auroc,4), "auprc": round(auprc,4), "f_measure": round(f_measure,4),
                "physio_accuracy": round(physio_accuracy,4),"utility_score": round(utility_score,4), 
                "positiveprediction": np.sum(y_predicted), 
            "baseline": round( (1 - round(y_test.mean().compute().loc['SepsisLabel'],2) )*100 , 2 )}
    print("\ny_test size:",y_test.shape, '1´s in y_test',y_test.sum())
    del  y_test
    print("Time spent in ",i,"th Fold iteration",round(time.time()-start1,2),"sec.\n")
    print("******************************************************************")
    return result

In [108]:
# Code for K-fold algorithm using multi processes
def KFold_patient_DASKMP(model, KforKFold=10,KforKNN=5, fillmethod=""):
    start = time.time()# time indicator for how long the Kfold func takes  
    print("Datasize",len(Uniq_ID) ,"with K Fold of ",KforKFold ," folds filling by KNN =",KforKNN)
    # initialisation of the array for storing the different intermediate results
    auroc_model = []
    auprc_model = []
    f_measure_model = []
    physio_accuracy_model = []
    utility_score_model = []
    positivepredictions = []
    baseline_model= []
    #The unique id sets have been created and shuffled in a fix manner in the third block, you can just use it here and no more any other manipulation
    idSets = np.array_split(Uniq_ID, KforKFold)# divide the ids into K groups

    flib.WritePatientIdsToFile(idSets, KforKFold)

#     OneFold_patient_MP(1, model, KforKNN,idSets)
    
    futures = []
    for i in range(0,KforKFold):
        print(i)
        f = client.submit(OneFold_patient_MP,i, model, KforKNN,idSets,fillmethod)
        futures.append(f)

    for f in futures:
        result = f.result()
        auroc_model.append(result["auroc"])
        auprc_model.append(result["auprc"])
        f_measure_model.append(result["f_measure"])
        physio_accuracy_model.append(result["physio_accuracy"])  
        utility_score_model.append(result["utility_score"])
        positivepredictions.append(result["positiveprediction"])
        baseline_model.append(result["baseline"])        
    print("\nEvaluation parameters of the utiltiy evaluation function:")
    print('Utility score of model:', utility_score_model)
    print('utility F1 of model:', f_measure_model)
    print('Utility accuracy of model:', physio_accuracy_model) 
    print('auroc of model:', auroc_model)
    print('auprc of model:', auprc_model)
    print("Positive Prediction: ", positivepredictions)
    print('Baseline model:', baseline_model)  
    KNN_auroc_mean.append(np.mean(auroc_model))
    KNN_auprc_mean.append(np.mean(auprc_model))
    KNN_F1Score_mean.append(np.mean(f_measure_model))
    KNN_F1Score_std.append(np.std(f_measure_model))
    KNN_accuracy_mean.append(np.mean(physio_accuracy_model))
    KNN_accuracy_std.append(np.std(physio_accuracy_model))
    KNN_UtilityScore_mean.append(np.mean(utility_score_model))
    KNN_UtilityScore_std.append(np.std(utility_score_model))
    KNN_positiveprediction_mean.append(np.mean(positivepredictions))
    KNN_baseline_mean.append(np.mean(baseline_model))
    
    totalTime=round(time.time()- start,2)
    KNN_total_Time.append(totalTime)
    print("\nTotal Time spent in  KFold function",totalTime,"sec.\n")
    Print_DATA()

In [109]:
#logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='saga', max_iter=1000,penalty='l1')
logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='lbfgs', max_iter=1000)
decisionTreeModel = tree.DecisionTreeClassifier(random_state=2)
randomForestModel = RandomForestClassifier(random_state=2)
XGBoostModel = XGBClassifier(random_state=2, n_jobs=4)
#AdaBoostModel = AdaBoostClassifier(random_state=2)

In [110]:
%%time
KFold_patient_DASKMP(decisionTreeModel,10,5)

Datasize 5000 with K Fold of  10  folds filling by KNN = 5
Time for writing id to file: 0.01
0
1
2
3
4
5
6
7
8
9

Evaluation parameters of the utiltiy evaluation function:
Utility score of model: [0.8754, 0.8736, 0.882, 0.8223, 0.8411, 0.8662, 0.8558, 0.8733, 0.8543, 0.8266]
utility F1 of model: [1.0, 0.9987, 0.9985, 0.9102, 0.965, 0.8947, 0.9959, 1.0, 0.9875, 0.9245]
Utility accuracy of model: [1.0, 0.9999, 0.9999, 0.9968, 0.9987, 0.9952, 0.9998, 1.0, 0.9995, 0.9974]
auroc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
auprc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Positive Prediction:  [336, 385, 332, 352, 360, 483, 368, 297, 396, 346]
Baseline model: [98.0, 98.0, 98.0, 98.0, 98.0, 98.0, 98.0, 98.0, 98.0, 98.0]

Total Time spent in  KFold function 35.45 sec.

Numbers of K: 1
KNN_UtilityScore_mean [0.8570599999999999]
KNN_UtilityScore_std [0.01994794224976601]
KNN_F1Score_mean [0.9675]
KNN_F1Score_std [0.03961297767146519]
KNN_auroc_mean [0.0]
KNN_auprc_mean [0.0]
KNN_accuracy_mean [0.99871

In [54]:
y_labels

NameError: name 'y_labels' is not defined

In [ ]:
# print("X_train size is: ",X_train.size * X_train.itemsize)
# print("X_test size is: ",X_test.size * X_test.itemsize)
# print("X_train size is: ",X_train.memory_usage(deep=True).sum().compute())
# print("X_test size is: ",X_test.memory_usage(deep=True).sum().compute())
# print("y_train size is: ",y_train.memory_usage(deep=True).sum().compute())
# print("y_test size is: ",y_test.memory_usage(deep=True).sum().compute())

In [46]:
def findBestKforKNN(model, KforKFold=10,KforKNNstart=1,KforKNNend=10, stepsize=5):
    if(KforKNNend<=1):
        print("K must be a interger larger than 1")
        return
    KNN_reset()
    step=stepsize
    for i in range(KforKNNstart,KforKNNend+1,step) : 
        print("KNN of K = ",i)    
        KFold_patient(model,10,i)

In [247]:
Print_DATA()

Numbers of K: 1
KNN_UtilityScore_mean [0.29173]
KNN_UtilityScore_std [0.2716681985437383]
KNN_F1Score_mean [0.29036]
KNN_F1Score_std [0.2419473339386074]
KNN_auroc_mean [0.0]
KNN_auprc_mean [0.0]
KNN_accuracy_mean [0.9420100000000001]
KNN_accuracy_std [0.0629201787982202]
KNN_positiveprediction_mean [82.9]
KNN_baseline_mean [97.8]
KNN_total_time [5.26]
